Note: To use `dropboxdol`, you'll need to have a dropbox access token. 
Additionally, that token should be allowed to do the operation that you are doing. 
For example, you only need the "sharing.write" permission to CREATE a (new) shared link. 

See more information in [Dropbox's Auth Guide](https://developers.dropbox.com/oauth-guide). 


By default, `dropboxdol` looks for the access tokens and other information is (well, really, the dropbox API) needs)
in the "default_dropbox_config.json" file of the dropboxdol app data. 
You can interact with those files via `dropboxdol.config_store`, which has a dict-like interface to the files in that folder. 



# Setup


By default, `dropboxdol` looks for the access tokens and other information is 
(well, really, the dropbox API) needs)
in the `"default_dropbox_config.json"` file of the dropboxdol app data. 
You can interact with those files via `dropboxdol.config_store`, which has a dict-like interface to the files in that folder. 

If you don't have a `"default_dropbox_config.json"` file, you'll need to specify the 
connection config explicitly (by specifying a config dict, a filepath, filename of the 
`config_store`).

Note that at this point, dropbox only dishes out temporary access tokens.
This means you can't just save an access token once and be over with it. 
So to get the convenience of automatic connections, you'll need to specify not only a 
`oauth2_access_token`, but also a `oauth2_refresh_token`, an `app_key` and an `app_secret`. 

You can read all about that annoying stuff in the [Dropbox's Auth Guide](https://developers.dropbox.com/oauth-guide).

But here's a few things to make it a bit less painful:
* make an "app" in the [app console](https://www.dropbox.com/developers/apps?_tk=pilot_lp&_ad=topbar4&_camp=myapps)
* specify scope and permissions you want on it
* note down the app key and the app secret

then use the following, which will walk you through the steps to get your access and refresh tokens.

These will then be stored in the `config_file` of your choice, so that all you have to 
do is mention the file to get the connection.


In [ ]:
from dropboxdol import create_config_file

create_config_file(
    config_file='NAME_OF_YOUR_APP_OR_WHATEVER_NAME_YOU_WILL_REMEMBER.json',
    app_key='YOUR_APP_KEY', 
    app_secret='YOUR_APP_SECRET',
)

If you already have a config file for this app, with app_key and app_secret, 
you can update the tokens by doing this:

In [ ]:
from dropboxdol import complete_config_file_with_refresh_token


complete_config_file_with_refresh_token(
    config_file='NAME_OF_YOUR_APP_OR_WHATEVER_YOU_CALLED_THAT_CONFIG.json',
)

If you want to edit some configs, you can do so by editing the file directly, or use 
`create_or_edit_config_file`.

In [1]:
from dropboxdol import create_or_edit_config_file

create_or_edit_config_file(
    config_file='CONFIG_FILE.json',
    # whatever edits you want to make... (specifying None will skip that config, leaving it unchanged)
    oauth2_access_token=None,
    oauth2_refresh_token=None,
    app_key=None,
    app_secret=None,
)

Then, assuming you have all your tokens in order, you can now do stuff...

# Get a dropbox "store"

In [1]:
from dropboxdol import DropboxFiles
from i2 import Sig 

drops = DropboxFiles('/Apps/py2store')

In [2]:
t = DropboxFiles('/Apps/py2store/py2store_data')
list(t)

['/test', '/test.txt', '/another_test.txt']

In [4]:
t['/test.txt']

b'This is a test.\nSee it work.\nAnd what about unicode? \xc3\xa8\xc3\xa9\xc3\xaa\xc3\xab\xc4\x93\xc4\x97\xc4\x99?'

# Get link of local file

In [22]:
from dropboxdol import dropbox_link

local_file = '/Users/thorwhalen/Dropbox/Apps/py2store/py2store_data/test.txt'

# Note: Your token needs the "sharing.write" permission to CREATE a (new) shared link
dropbox_url = dropbox_link(local_file)
print(dropbox_url)

https://www.dropbox.com/scl/fi/3o8ooqje4f497npxdeiwg/test.txt?rlkey=x9jsd8u7k147x6fzc7stxozqe&dl=0


In [ ]:
from functools import partial

# if you want to talk "relative" to the dropbox root dir, do this:
my_dropbox_link = partial(dropbox_link, dropbox_local_rootdir='/Users/thorwhalen/Dropbox')
# If you want a "direct (download) link", do this:
dl1_link = my_dropbox_link('Apps/py2store/py2store_data/test.txt', dl=1)
dl1_link

'https://www.dropbox.com/scl/fi/3o8ooqje4f497npxdeiwg/test.txt?rlkey=x9jsd8u7k147x6fzc7stxozqe&dl=1'

## Get store from link

In [1]:
from dropboxdol import DropboxLinkReaderWithToken

s = DropboxLinkReaderWithToken(
    url="https://www.dropbox.com/sh/0ru09jmk0w9tdnr/AAA-PPON2sYmwUUoGQpBQh1Ia?dl=0"
)
keys = list(s)
keys

['inner_folder',
 'b1467f55540c4695bf483bc542e43256',
 '0b98e2af76c94a0a9cc2808866dd62de',
 '3372aa35ea444c758bfa2e4599b2576d',
 '9de9d98a4c4648cca1bc1131c307a365',
 '91c744890d374dd8bc914f1153311b0c',
 '57af886dd22f4a23a678a3de3eb996a0',
 '43ba127e5e9245ec983c9f39e4ed7306']

In [ ]:
from dropboxdol import DropboxLinkReaderWithToken

s = DropboxLinkReaderWithToken(
    url="https://www.dropbox.com/sh/0ru09jmk0w9tdnr/AAA-PPON2sYmwUUoGQpBQh1Ia?dl=0"
)
raw_keys = list(s._raw_iter())
raw_keys

[FolderMetadata(id='id:YlSl2LVZjVAAAAAAAAAAGg', name='inner_folder', parent_shared_folder_id='1067644886', path_display=NOT_SET, path_lower=NOT_SET, preview_url=NOT_SET, property_groups=NOT_SET, shared_folder_id=NOT_SET, sharing_info=FolderSharingInfo(no_access=False, parent_shared_folder_id='1067644886', read_only=True, shared_folder_id=NOT_SET, traverse_only=False)),
 FileMetadata(client_modified=datetime.datetime(2019, 9, 20, 12, 40, 12), content_hash='7267b55693d544863f5ee220b4e823f352f96f9baa76319a802cf91965b2f70d', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:YlSl2LVZjVAAAAAAAAAACw', is_downloadable=True, media_info=NOT_SET, name='b1467f55540c4695bf483bc542e43256', parent_shared_folder_id='1067644886', path_display=NOT_SET, path_lower=NOT_SET, preview_url=NOT_SET, property_groups=NOT_SET, rev='592fb5f0b35f13fa2f7d6', server_modified=datetime.datetime(2019, 9, 20, 12, 40, 12), sharing_info=FileSharingInfo(modified_by='dbid:AAD2NjNI7OPt5Q

In [14]:
t.name

'inner_folder'

In [28]:
type(s.shared_link)

dropbox.files.SharedLink

In [30]:
import os
import dropbox
from dropbox.exceptions import ApiError
from dropbox.files import SharedLink

def list_files_in_shared_link(shared_link: str, path: str = "", client: dropbox.Dropbox = None) -> list:
    """
    List all files and folders under a Dropbox shared link folder.

    Parameters:
      shared_link (str): The URL of the Dropbox shared link (expected to point to a folder).
      path (str): Optional path within the shared link folder (default is the root of the shared link).
      client (dropbox.Dropbox, optional): An instantiated Dropbox client. If not provided, the function
                                            will attempt to create one using the environment variable
                                            DROPBOX_ACCESS_TOKEN.

    Returns:
      list: A list of metadata entries (files and folders) found under the specified shared link.

    Raises:
      Exception: If no client is provided and DROPBOX_ACCESS_TOKEN is not set, or if an API error occurs.
    """
    # Create a Dropbox client if one wasn't provided.
    client = get_client()
    

    # Create a SharedLink object from the provided URL.
    shared_link_obj = SharedLink(url=shared_link)

    try:
        # List the folder at the specified path within the shared link.
        result = client.files_list_folder(path=path, shared_link=shared_link_obj)
    except ApiError as e:
        raise Exception(f"Error listing folder for shared link: {e}")

    entries = result.entries[:]
    # If there are more entries, continue to page through the results.
    while result.has_more:
        result = client.files_list_folder_continue(result.cursor)
        entries.extend(result.entries)
    return entries

# Example usage:
if __name__ == "__main__":
    # Replace with your shared link URL.
    url="https://www.dropbox.com/sh/0ru09jmk0w9tdnr/AAA-PPON2sYmwUUoGQpBQh1Ia?dl=0"
    shared_link_url = url
    try:
        files = list_files_in_shared_link(shared_link_url)
        for entry in files:
            print(f"{entry.name} - {entry.path_lower}")
    except Exception as e:
        print("Error:", e)

inner_folder - None
b1467f55540c4695bf483bc542e43256 - None
0b98e2af76c94a0a9cc2808866dd62de - None
3372aa35ea444c758bfa2e4599b2576d - None
9de9d98a4c4648cca1bc1131c307a365 - None
91c744890d374dd8bc914f1153311b0c - None
57af886dd22f4a23a678a3de3eb996a0 - None
43ba127e5e9245ec983c9f39e4ed7306 - None


In [3]:
import json
import os
from dropboxdol.tests.test_dropbox import test_config
from dropboxdol import DropboxPersister

s = DropboxPersister('/py2store_data/test/', connection_config=test_config)

# >>> if '/py2store_data/test/_can_remove' in s:
# ...     del s['/py2store_data/test/_can_remove']
# ...
# >>>
# >>> n = len(s)
# >>> if n == 1:
# ...     assert list(s) == ['/py2store_data/test/_can_remove']
# ...
# >>> s['/py2store_data/test/_can_remove'] = b'this is a test'
# >>> assert len(s) == n + 1
# >>> assert s['/py2store_data/test/_can_remove'] == b'this is a test'
# >>> '/py2store_data/test/_can_remove' in s
# True
# >>> del s['/py2store_data/test/_can_remove']

In [4]:
list(s)

['/py2store_data/test/file_in_test_folder.txt',
 '/py2store_data/test/another_file_in_test_folder.txt']

# Scrap

In [5]:
from dropboxdol.util import config_store

list(config_store)


['default_dropbox_config.json']